In [8]:
#import packages
import sys
import glob, os

import torch.nn as nn
import torch.optim as optim

# set source path
codepath = os.path.abspath("/Users/Pavel/Documents/repos/machine-learning/stepik-deep-learning/16-HW-semantic-segmentation/UNet")
if codepath not in sys.path:
    sys.path.append(codepath)

#automatically track changes in the source code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from data_handling.base import BaseDataLoader
from UNet.models.unet import UNet
from UNet.training.base_trainer import BaseTrainer

In [11]:
# Create the corresponding dataloader for training and validation
data_loader = BaseDataLoader(dataset=cifar10_dataset,
                             batch_size=16,
                             validation_split=0.1)

NameError: name 'BaseDataLoader' is not defined

In [4]:
# Define the model
model = UNet()
save_dir = os.getcwd()+'\\runs\\exp1'

In [9]:
# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
# Initialize the trainer
trainer = BaseTrainer(model=model,
                      criterion=criterion,
                      optimizer=optimizer,
                      data_loader=data_loader,
                      n_epochs=100,
                      save_dir=save_dir,
                      save_model_frequency=1)

NameError: name 'data_loader' is not defined

In [1]:
# inheritance " in an old-fashioned and unflexible way  " - don't use it

class Female_Grandparent:
    def __init__(self):
        self.grandma_name = 'Grandma'

class Male_Grandparent:
    def __init__(self):
        self.grandpa_name = 'Grandpa'

class Parent(Female_Grandparent, Male_Grandparent):
    def __init__(self):
        Female_Grandparent.__init__(self)
        Male_Grandparent.__init__(self)

        self.parent_name = 'Parent Class'

class Child(Parent):
    def __init__(self):
        Parent.__init__(self)
        
        #---------------------------------------------------------------------------------------#
        for cls in Parent.__bases__: # This block grabs the classes of the child
             cls.__init__(self)      # class (which is named 'Parent' in this case), 
                                     # and iterates through them, initiating each one.
                                     # The result is that each parent, of each child,
                                     # is automatically handled upon initiation of the 
                                     # dependent class. WOOT WOOT! :D
        #---------------------------------------------------------------------------------------#


In [2]:
g = Female_Grandparent()
print (g.grandma_name)

p = Parent()
print (p.grandma_name)

child = Child()

print (child.grandma_name)

Grandma
Grandma
Grandma


In [3]:
# comparison of inheritance whilst using super and not using super

class SomeBaseClass(object):
    def __init__(self):
        print('SomeBaseClass.__init__(self) called')
    
class UnsuperChild(SomeBaseClass):
    def __init__(self):
        print('UnsuperChild.__init__(self) called')
        SomeBaseClass.__init__(self)
    
class SuperChild(SomeBaseClass):
    def __init__(self):
        print('SuperChild.__init__(self) called')
        super().__init__()


In [4]:
# add another class iheriting from the base class
class InjectMe(SomeBaseClass):
    def __init__(self):
        print('InjectMe.__init__(self) called')
        super().__init__()

        
# Now you want to have a class  inheriting from the newly created InjectMe and the UnsuperChild
# Both inherit from the Base class
# InjectMe uses super, but UnsuperChild hard-codes the inheritance from the BaseCLass
class UnsuperInjector(UnsuperChild, InjectMe): pass

# Now you want to have a class  inheriting from the newly created InjectMe and the SuperChild
# Both inherit from the Base class
# Both InjectMe and SuperChild use super to inherit from the BaseCLass
class SuperInjector(SuperChild, InjectMe): pass

In [5]:
# Using the un-super child fails to inject the dependency 
# because the child you're using has hard-coded the method to be called after its own:
o = UnsuperInjector()

UnsuperChild.__init__(self) called
SomeBaseClass.__init__(self) called


In [6]:
# However, the class with the child that uses super can correctly inject the dependency:
o2 = SuperInjector()

SuperChild.__init__(self) called
InjectMe.__init__(self) called
SomeBaseClass.__init__(self) called


# tests 

In [8]:
import torch
import torchvision
import torch.nn as nn


#from UNet.models.base import BaseNet


class UNet:
    def __init__(self):
        super().__init__()
        # encoder (downsampling)
        # Each enc_conv/dec_conv block should look like this:
        # nn.Sequential(
        #     nn.Conv2d(...),
        #     ... (2 or 3 conv layers with relu and batchnorm),
        # )
        ##################
        # encoder layer 0
        #################
        # 3, 572, 572
        self.e0_conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3),
            # 64, 570, 570
            nn.Conv2d(64, 64, kernel_size=3),
            # 64, 568, 568
            nn.BatchNorm2d(64),
            nn.ReLU()
                    )

In [ ]:
nn.Conv2d()